# [DeepSphere]: a spherical convolutional neural network
[DeepSphere]: https://github.com/SwissDataScienceCenter/DeepSphere

[Nathanaël Perraudin](https://perraudin.info), [Michaël Defferrard](http://deff.ch), Tomasz Kacprzak, Raphael Sgier

# Demo regression

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import shutil

# Run on CPU.
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import healpy as hp
import tensorflow as tf

from deepsphere import models, experiment_helper, plot
from deepsphere.data import LabeledDataset
from deepsphere.utils import HiddenPrints

In [ ]:
plt.rcParams['figure.figsize'] = (17, 5)

In [ ]:
EXP_NAME = 'regression'

## 1 Data loading

Let us create a very simple syntetic dataset by simply filtering random noise with different Gaussian filters.

In [ ]:
Nside=32
Nsamples = 1000

In [ ]:
def arcmin2rad(x):
    return x / 60 / 360 * 2 * np.pi
def gaussian_smoothing(sig, sigma, nest=True):
    if nest:
        sig = hp.reorder(sig, n2r=True)
    smooth = hp.sphtfunc.smoothing(sig, sigma=sigma)
    if nest:
        smooth = hp.reorder(smooth, r2n=True)
    smooth/=np.linalg.norm(smooth)
    return smooth

In [ ]:
x = np.random.randn(Nsamples, hp.nside2npix(Nside))
t = np.random.rand(Nsamples,2)+0.5

with HiddenPrints():
    xs1 = np.array(list(map(gaussian_smoothing, x,t[:,0]/Nside)))    
    xs2 = np.array(list(map(gaussian_smoothing, x,t[:,1]/Nside)))
xs = np.stack((xs1,xs2), axis=2)

Let us plot a map of each class. It is not simple to visually catch the differences.

In [ ]:
cm = plt.cm.RdBu_r
cm.set_under('w')
hp.mollview(xs[0,:,0], title='Smooth', nest=True, cmap=cm)
hp.mollview(x[0], title='Original noise', nest=True, cmap=cm)

## 2 Data preparation

Let us split the data into training and testing sets. The raw data is stored into `x_raw` and the power spectrum densities into `x_psd`.

In [ ]:
# Normalize and transform the data, i.e. extract features.
x_raw = xs / np.mean(xs**2) # Apply some normalization (We do not want to affect the mean)

# Create the label vector
labels = t-0.5
# Random train / test split
ntrain = 800
x_raw_train = x_raw[:ntrain]
x_raw_test = x_raw[ntrain:]
labels_train = labels[:ntrain]
labels_test = labels[ntrain:]


## Regression using DeepSphere

In [ ]:
params = dict()
params['dir_name'] = EXP_NAME

# Types of layers.
params['conv'] = 'chebyshev5'  # Graph convolution: chebyshev5 or monomials.
params['pool'] = 'max'  # Pooling: max or average.
params['activation'] = 'relu'  # Non-linearity: relu, elu, leaky_relu, softmax, tanh, etc.
params['statistics'] = None  # Statistics (for invariance): None, mean, var, meanvar, hist.

# Architecture.

params['statistics'] = 'mean' # We build an invariance for the problem
params['nsides'] = [Nside, Nside//2, Nside//4, Nside//8]  # Pooling: number of pixels per layer.
params['F'] = [16, 32, 64]  # Graph convolutional layers: number of feature maps.
params['M'] = [64, 2]  # Fully connected layers: output dimensionalities.
params['loss'] = 'l2'
params['input_channel'] = 2
params['K'] = [5] * len(params['F'])  # Polynomial orders.
params['batch_norm'] = [True] * len(params['F'])  # Batch normalization.

# Regularization.
params['regularization'] = 0  # Amount of L2 regularization over the weights (will be divided by the number of weights).
params['dropout'] = 1  # Percentage of neurons to keep.

# Training.
params['num_epochs'] = 10  # Number of passes through the training data.
params['batch_size'] = 16  # Number of samples per training batch. Should be a power of 2 for greater speed.
params['eval_frequency'] = 15  # Frequency of model evaluations during training (influence training time).
params['scheduler'] = lambda step: 1e-4  # Constant learning rate.
params['optimizer'] = lambda lr: tf.train.GradientDescentOptimizer(lr)
#params['optimizer'] = lambda lr: tf.train.MomentumOptimizer(lr, momentum=0.5)
# params['optimizer'] = lambda lr: tf.train.AdamOptimizer(lr, beta1=0.9, beta2=0.99, epsilon=1e-8)

In [ ]:
model = models.deepsphere(**params)

In [ ]:
# Cleanup before running again.
shutil.rmtree('summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

In [ ]:
training = LabeledDataset(x_raw_train, labels_train)
testing = LabeledDataset(x_raw_test, labels_test)

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step = model.fit(training, testing)

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

In [ ]:
pred_test = model.predict(x_raw_test)
pred_train = model.predict(x_raw_train)


In [ ]:
np.mean(np.abs(pred_test-labels_test))/np.std(labels_test)

In [ ]:
plt.plot(labels_train[:,0], pred_train[:,0], 'o', label='Train')
plt.plot(labels_test[:,0], pred_test[:,0], 'o', label='Test')
plt.plot([0,1],[0,1])
plt.legend()

In [ ]:
plt.plot(labels_train[:,1], pred_train[:,1], 'o', label='Train')
plt.plot(labels_test[:,1], pred_test[:,1], 'o', label='Test')
plt.plot([0,1],[0,1])
plt.legend()